In [ ]:
import numpy as np
from sklearn.datasets import make_moons
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
import h5py

In [ ]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

In [ ]:
def d_sigmoid(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

In [ ]:
def relu(Z):
    A = np.maximum(0, Z)
    cache = Z
    return A, cache

In [ ]:
def d_relu(dA, cache):
    Z = cache
    dZ = np.array(dA, copy = True)
    
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    
    return dZ

In [ ]:
def init_params(layers):
    
    np.random.seed(2)
    
    params = {}
    
    L = len(layers)
    
    for l in range(1,L):
        params['W' + str(l)] = np.random.randn(layers[l], layers[l-1]) * 0.01
        
        params['b' + str(l)] = np.zeros((layers[l], 1))
        
    return params

In [ ]:
def forward_prop(X, params, activations):
    L = len(params) // 2
    caches = []
    A = X
    for l in range(L):
        A_prev = A
        A, cache = linear_activation_forward(l+1, A_prev, params['W'+str(l+1)], params['b'+str(l+1)], activations[l+1]) 
        caches.append(cache)    
    return A, caches

In [ ]:
def linear_activation_forward(l, A, W, b, activation):
    linear_cache = {}
    activation_cache = {}
    Z, linear_cache = linear_forward(A, W, b)
    
    if activation == 'relu':
        A, activation_cache = relu(Z)
    
    elif activation == 'sigmoid':
        A, activation_cache = sigmoid(Z)     
    
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [ ]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A,W,b)
    return Z, cache

In [ ]:
def calc_cost(A, y):
    m = y.shape[1]
    cost = -(1/m) * np.sum(np.dot(y, np.log(A).T) + np.dot((1-y), np.log(1-A).T))

    return cost

In [ ]:
def back_prop(A, y, cache, activations):
    L = len(cache)
    grads = {}
    dAL = -(np.divide(y, A)  - np.divide((1-y), (1-A)))
    
    current_cache = cache[L-1]
    
    grads['dA'+str(L-1)], grads['dW'+str(L)], grads['db'+str(L)] = linear_activation_back(dAL, current_cache, activations[L])
            
    
    for l in reversed(range(L-1)):
        current_cache = cache[l]
        grads['dA'+str(l)], grads['dW'+str(l+1)], grads['db'+str(l+1)] = linear_activation_back(grads['dA'+str(l+1)], current_cache, activations[l+1])
        
    
    return grads

In [ ]:
def linear_activation_back(dA, cache, activation):
    
    if activation == 'relu':
        dZ = d_relu(dA, cache[1])
    elif activation == 'sigmoid':
        dZ = d_sigmoid(dA, cache[1])    
    
    dA_prev, dW, db = linear_back(dZ, cache[0])
    
    return dA_prev, dW, db
    

In [ ]:
def linear_back(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dA_prev = np.dot(W.T, dZ) 
    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis = 1, keepdims = True)
    
    return dA_prev, dW, db

In [ ]:
def update_params(parameters, grads, learning_rate):
    L = len(parameters) // 2
    for l in range(1, L+1):
        parameters['W'+str(l)] = parameters['W'+str(l)] - learning_rate*grads['dW'+str(l)]
        parameters['b'+str(l)] = parameters['b'+str(l)] - learning_rate*grads['db'+str(l)]
    return parameters

In [ ]:
def predict(x_test, params, activations):
    L = len(params) // 2
    A, cache = forward_prop(x_test, params, activations)
    
    predictions = (A >= 0.5).astype(int)
    
    return predictions

In [ ]:
def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    y = y.reshape(y.shape[1])
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)

In [ ]:
def accuracy(predictions, y_test):
    x = (np.abs(predictions - y_test)).sum()
    acc = 100 - ((x/y_test.shape[1]) * 100)
    return str(acc) + "%"

In [ ]:
def nn_model(X, Y, learning_rate, num_iterations, layers, activation_functions):
    params = init_params(layers)
    for i in range(num_iterations):
        AL, caches = forward_prop(X, params, activation_functions)
        cost = calc_cost(AL, Y)
        if(i%100 == 0):
            print(cost)
        grads = back_prop(AL, Y, caches, activations)    
        params = update_params(params, grads, learning_rate)
        
    return params

In [ ]:
dataset = make_moons(noise=0.3, random_state=2)

X = dataset[0]
Y = dataset[1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)

y_test = y_test.reshape(1, y_test.shape[0])
y_train = y_train.reshape(1, y_train.shape[0])

X_train = X_train.T
X_test = X_test.T

In [ ]:
X, Y = dataset
X, Y = X.T, Y.reshape(Y.shape[0])

# Visualize the data
plt.scatter(X[0, :], X[1, :], c = Y);

In [ ]:
layers = [2, 4, 3, 1]
activations = ['none','relu', 'relu', 'sigmoid']

In [ ]:
params = nn_model(X_train, y_train, 0.4, 3000, layers, activations)

In [ ]:
predictions = predict(X_test, params, activations)

In [ ]:
predictions.shape

In [ ]:
plot_decision_boundary(lambda x: predict(x.T, params, activations), X_test, y_test)

In [ ]:
accuracy(predictions, y_test)

## Load cat dataset

In [ ]:
file_name = 'train_catvnoncat.h5'

In [ ]:
with h5py.File(file_name, 'r') as f:
    # List all groups
    print("Keys: %s" % f.keys())
    b_group_key = list(f.keys())[1]
    c_group_key = list(f.keys())[2]
    a_group_key = list(f.keys())[0]

    # Get the data
    x_train = list(f[b_group_key])
    y_train = list(f[c_group_key])
    labels = list(f[a_group_key])

In [ ]:
X_train  = np.array(x_train)
Y_train  = np.array(y_train)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], -1)).T
Y_train = Y_train.reshape((Y_train.shape[0],1)).T

In [ ]:
X_train = X_train / 255
Y_train = Y_train

In [ ]:
X_train.shape


In [ ]:
Y_train.shape


In [ ]:
file_name = 'test_catvnoncat.h5'


In [ ]:
with h5py.File(file_name, 'r') as f:
    # List all groups
    print("Keys: %s" % f.keys())
    b_group_key = list(f.keys())[1]
    c_group_key = list(f.keys())[2]
    a_group_key = list(f.keys())[0]

    # Get the data
    x_test = list(f[b_group_key])
    y_test = list(f[c_group_key])
    labels_test = list(f[a_group_key])

In [ ]:
X_test = np.array(x_test)
Y_test = np.array(y_test)

In [ ]:
X_test = X_test.reshape((X_test.shape[0], -1)).T
Y_test = Y_test.reshape((Y_test.shape[0],1)).T

In [ ]:

X_test = X_test / 255
Y_test = Y_test

In [ ]:
X_test.shape


In [ ]:
Y_test.shape


In [ ]:
layers = [X_train.shape[0], 20, 7, 5, 1]
activations = ['input', 'relu','relu', 'relu', 'sigmoid']

In [ ]:
params = nn_model(X_train, Y_train, 0.0075, 2500, layers, activations)

In [ ]:
predictions = predict(X_test, params, act_funcs)

In [ ]:
accuracy(predictions, Y_test)